# dMtb interactive glimpse maker

In [72]:
import napari
from cellpose import models
from macrohet import dataio, tile, visualise, tools, aws_tools
import numpy as np
import os

def view(img):
    return napari.Viewer().add_image(img)

from napari_animation import Animation
from tqdm.notebook import tqdm

import btrack
import dask.array as da

from skimage.transform import rescale, resize, downscale_local_mean

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="ticks")
sns.set_palette("Spectral")


### Load all metadata

Both the image metadata and the assay layout metadata

In [2]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet/'
metadata_fn = os.path.join(base_dir, 'macrohet_images/PS0000/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [15]:
%%time
metadata_path = os.path.join(base_dir, 'macrohet_images/PS0000/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)

Reading metadata XML file...
Extracting metadata complete!
CPU times: user 13.1 ms, sys: 94 µs, total: 13.2 ms
Wall time: 12 ms


Add technical replicate label to assay layout

In [16]:
assay_layout['Technical replicate'] = assay_layout.groupby(['Strain', 'Compound', 'Concentration', 'ConcentrationEC']).cumcount() + 1
assay_layout

Strain Compound Concentration ConcentrationEC  Technical replicate
3 4    RD1     CTRL             0             EC0                    1
  5     WT     CTRL             0             EC0                    1
  6     WT      PZA            60            EC50                    1
  7     WT      RIF           0.1            EC50                    1
  8     WT      INH          0.04            EC50                    1
  9     WT      BDQ          0.02            EC50                    1
4 4    RD1     CTRL             0             EC0                    2
  5     WT     CTRL             0             EC0                    2
  6     WT      PZA            60            EC50                    2
  7     WT      RIF           0.1            EC50                    2
  8     WT      INH          0.04            EC50                    2
  9     WT      BDQ          0.02            EC50                    2
5 4    RD1     CTRL             0             EC0                    3
  5     WT     CTRL             0             EC0                    3
  6     WT      PZA           400            EC99                    1
  7     WT      RIF             2            EC99                    1
  8     WT      INH             2            EC99                    1
  9     WT      BDQ           2.5            EC99                    1
6 4    RD1     CTRL             0             EC0                    4
  5     WT     CTRL             0             EC0                    4
  6     WT      PZA           400            EC99                    2
  7     WT      RIF             2            EC99                    2
  8     WT      INH             2            EC99                    2
  9     WT      BDQ           2.5            EC99                    2

# Iteratively load all tracks

and append to a track_dict dictionary

In [4]:
tracks_dict = dict()
### iterate over all experimental conditions
for (row, column), info in tqdm(assay_layout.iterrows(), 
                                desc = 'Progress through positions',
                                total = len(assay_layout)):

    ### load tracks
    with btrack.io.HDF5FileHandler(os.path.join(base_dir, 
                                                f'labels/macrohet_seg_model/{row,column}.h5'), 
                                       'r', 
                                       obj_type='obj_type_1'
                                       ) as reader:
        tracks = reader.tracks
            
    ### append tracks to dictionary
    tracks_dict[(row, column)] = tracks

Progress through positions:   0%|          | 0/24 [00:00<?, ?it/s]

[INFO][2023/06/19 03:38:53 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5...
[INFO][2023/06/19 03:38:53 PM] Loading tracks/obj_type_1
[INFO][2023/06/19 03:38:53 PM] Loading LBEP/obj_type_1
[INFO][2023/06/19 03:38:53 PM] Loading objects/obj_type_1 (39878, 5) (39878 filtered: None)
[INFO][2023/06/19 03:38:53 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5
[INFO][2023/06/19 03:38:53 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 5).h5...
[INFO][2023/06/19 03:38:53 PM] Loading tracks/obj_type_1
[INFO][2023/06/19 03:38:53 PM] Loading LBEP/obj_type_1
[INFO][2023/06/19 03:38:53 PM] Loading objects/obj_type_1 (41424, 5) (41424 filtered: None)
[INFO][2023/06/19 03:38:54 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 5).h5
[INFO][2023/06/19 03:38:54 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 6).h5...
[INFO][2023/06/19 03:38:54 PM] Loading tracks/obj_type_1
[INFO]

[INFO][2023/06/19 03:39:11 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 5).h5
[INFO][2023/06/19 03:39:11 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 6).h5...
[INFO][2023/06/19 03:39:11 PM] Loading tracks/obj_type_1
[INFO][2023/06/19 03:39:11 PM] Loading LBEP/obj_type_1
[INFO][2023/06/19 03:39:11 PM] Loading objects/obj_type_1 (45688, 5) (45688 filtered: None)
[INFO][2023/06/19 03:39:12 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 6).h5
[INFO][2023/06/19 03:39:12 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 7).h5...
[INFO][2023/06/19 03:39:12 PM] Loading tracks/obj_type_1
[INFO][2023/06/19 03:39:12 PM] Loading LBEP/obj_type_1
[INFO][2023/06/19 03:39:12 PM] Loading objects/obj_type_1 (45214, 5) (45214 filtered: None)
[INFO][2023/06/19 03:39:14 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 7).h5
[INFO][2023/06/19 03:39:14 PM] Opening HDF file: /mnt/DATA/macrohe

# Compile all longer length tracks into dataframe

Add extra information such as the MSD of cells between frames

In [5]:
# df = pd.read_pickle('/mnt/DATA/macrohet/results/preliminary_sc_measures/sc_dfs/sc_df_>40.pkl')
df = pd.read_pickle(os.path.join(base_dir, 'results/preliminary_sc_measures/sc_dfs/sc_df_GT_70_interp.pkl'))
df

,Time (hours),x,y,x scaled,y scaled,Infection status,Initial infection status,Final infection status,Area,Intracellular mean Mtb content,...,delta Mtb/dt,Eccentricity,MSD,Strain,Compound,Concentration,Technical replicate,Cell ID,Acquisition ID,Unique ID
0,0,457.172943,8.791715,2304.151633,44.310242,1.0,1.0,0.0,20542.0,343.157196,...,0.346945,0.929721,0.000000,RD1,CTRL,EC0,1,426,"(3, 4)",426.3.4
1,1,459.138947,6.718642,2314.060291,33.861954,0.0,1.0,0.0,16683.0,344.687042,...,0.346945,0.977788,2.857062,RD1,CTRL,EC0,1,426,"(3, 4)",426.3.4
2,2,460.555237,10.785886,2321.198394,54.360864,0.0,1.0,0.0,18935.0,343.875061,...,0.346945,0.733415,4.306780,RD1,CTRL,EC0,1,426,"(3, 4)",426.3.4
3,3,455.878815,11.143067,2297.629226,56.161059,1.0,1.0,0.0,19937.0,345.565796,...,0.346945,0.685354,4.690043,RD1,CTRL,EC0,1,426,"(3, 4)",426.3.4
4,4,462.634186,14.050420,2331.676296,70.814116,1.0,1.0,0.0,21793.0,350.585083,...,0.346945,0.727210,7.354437,RD1,CTRL,EC0,1,426,"(3, 4)",426.3.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296992,70,628.562500,1099.782104,3167.955000,5542.901807,1.0,0.0,1.0,101991.0,377.731873,...,-0.025779,0.697015,5.203749,WT,BDQ,EC99,2,935,"(6, 9)",935.6.9
296993,71,628.463684,1102.144409,3167.456968,5554.807822,1.0,0.0,1.0,100978.0,377.812103,...,-0.025779,0.690471,2.364371,WT,BDQ,EC99,2,935,"(6, 9)",935.6.9
296994,72,628.672058,1101.594727,3168.507173,5552.037422,1.0,0.0,1.0,108397.0,376.812805,...,-0.025779,0.665902,0.587853,WT,BDQ,EC99,2,935,"(6, 9)",935.6.9
296995,73,630.372803,1102.817749,3177.078926,5558.201455,1.0,0.0,1.0,107079.0,377.437073,...,-0.025779,0.647429,2.094831,WT,BDQ,EC99,2,935,"(6, 9)",935.6.9


### Renaming delta Mtb method of choice for ease of use

In [12]:
df['dMtb'] = df['delta Mtb max fold-change normalised max area']

In [9]:
for index, row_ in tqdm(df.iterrows(), total =len(df)):
    acquisition_id = row_['Acquisition ID']
    repeat_id = assay_layout.loc[acquisition_id]['Technical replicate']
    df.at[index, 'Technical replicate'] = repeat_id

  0%|          | 0/296997 [00:00<?, ?it/s]

### New style

#### What do the different distributions of deltaMtb look like?

In [18]:
df_delta_mtb = df.drop_duplicates(subset=['delta Mtb raw', 'delta Mtb max raw','delta Mtb max smooth', 'delta Mtb max fold-change'])
df_delta_mtb

,Time (hours),x,y,x scaled,y scaled,Infection status,Initial infection status,Final infection status,Area,Intracellular mean Mtb content,...,Eccentricity,MSD,Strain,Compound,Concentration,Technical replicate,Cell ID,Acquisition ID,Unique ID,dMtb
0,0,457.172943,8.791715,2304.151633,44.310242,1.0,1.0,0.0,20542.0,343.157196,...,0.929721,0.0,RD1,CTRL,EC0,1,426,"(3, 4)",426.3.4,0.000586
74,0,663.630920,10.924337,3344.699839,55.058660,0.0,0.0,0.0,20098.0,353.067780,...,0.805467,0.0,RD1,CTRL,EC0,1,427,"(3, 4)",427.3.4,0.000544
149,0,725.798340,20.058319,3658.023633,101.093928,1.0,1.0,0.0,29683.0,362.856537,...,0.540146,0.0,RD1,CTRL,EC0,1,435,"(3, 4)",435.3.4,-0.000789
224,0,89.238220,86.176323,449.760630,434.328668,0.0,0.0,0.0,29980.0,368.084900,...,0.676988,0.0,RD1,CTRL,EC0,1,468,"(3, 4)",468.3.4,0.000000
299,0,331.759827,89.224770,1672.069526,449.692839,1.0,1.0,1.0,25356.0,400.908905,...,0.700553,0.0,RD1,CTRL,EC0,1,472,"(3, 4)",472.3.4,-0.000010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296635,2,1090.928833,529.117737,5498.281318,2666.753394,0.0,0.0,0.0,13978.0,376.818298,...,0.883639,0.0,WT,BDQ,EC99,2,817,"(6, 9)",817.6.9,0.356070
296708,2,440.705811,626.353271,2221.157285,3156.820488,1.0,1.0,1.0,61767.0,398.949249,...,0.971427,0.0,WT,BDQ,EC99,2,814,"(6, 9)",814.6.9,0.000030
296781,2,1139.693848,723.973694,5744.056992,3648.827417,1.0,1.0,1.0,33122.0,410.231384,...,0.871169,0.0,WT,BDQ,EC99,2,740,"(6, 9)",740.6.9,-0.003271
296854,3,10.119422,1189.885742,51.001887,5997.024141,1.0,1.0,0.0,10593.0,354.040588,...,0.745716,0.0,WT,BDQ,EC99,2,865,"(6, 9)",865.6.9,-0.002747


# Create outlier df using same approach as in dMtb interactive plot making notebook

In [34]:
outliers = list()
fns = list()
for index, condition in df_delta_mtb.groupby(['Strain', 'Compound', 'Concentration']).size().reset_index().rename(columns={0: 'count'}).iterrows():

    strain = condition['Strain']
    compound = condition['Compound']
    concentration = condition['Concentration']
    label = f'{strain}, {compound}, {concentration}'
    subset_df = df_delta_mtb[(df_delta_mtb['Strain'] == strain)
                             & (df_delta_mtb['Compound'] == compound)
                             & (df_delta_mtb['Concentration'] == concentration)]
#                              & (df_delta_mtb['Technical replicate'] == 1)
   
    # create a separate outlier df for below plot
    # Calculate the mean and standard deviation of dMtb
    mean = subset_df['dMtb'].mean()
    std = subset_df['dMtb'].std()

    # Define a threshold to determine outliers (e.g., 2 standard deviations from the mean)
    threshold = 1 * std

    # Filter out the non outliers from subset_df
    outlier_df = subset_df[abs(subset_df['dMtb'] - mean) >= threshold]
    outliers.append(outlier_df)
    
    # append url to df 
    for index, row_ in outlier_df.iterrows():
        unique_ID = row_['Unique ID']
        cell_ID, row, column = unique_ID.split('.') 
        t0 = row_['Time (hours)']
        outlier_df.at[index, 'glimpse_url_t0'] = f'https://macrohet.s3.eu-west-2.amazonaws.com/dMtb/dMtb_glimpses/glimpse_{cell_ID}.{row}.{column}_t{t0}.png'
        outlier_df.at[index, 'glimpse_url_tn'] = f'https://macrohet.s3.eu-west-2.amazonaws.com/dMtb/dMtb_glimpses/glimpse_{cell_ID}.{row}.{column}_t{-1}.png'
        
        # glimpse fns
        fns.append(f'glimpse_{cell_ID}.{row}.{column}_t{t0}.png')
        fns.append(f'glimpse_{cell_ID}.{row}.{column}_t{-1}.png')
        
# concat dfs together
outliers = pd.concat(outliers, ignore_index=True)
outliers

,Time (hours),x,y,x scaled,y scaled,Infection status,Initial infection status,Final infection status,Area,Intracellular mean Mtb content,...,Strain,Compound,Concentration,Technical replicate,Cell ID,Acquisition ID,Unique ID,dMtb,glimpse_url_t0,glimpse_url_tn
0,0,358.422546,340.802887,1806.449634,1717.646550,0.0,0.0,0.0,23559.0,363.705963,...,RD1,CTRL,EC0,1,599,"(3, 4)",599.3.4,0.037397,https://macrohet.s3.eu-west-2.amazonaws.com/dM...,https://macrohet.s3.eu-west-2.amazonaws.com/dM...
1,0,991.728455,642.783386,4998.311411,3239.628267,0.0,0.0,0.0,60327.0,373.826721,...,RD1,CTRL,EC0,1,103,"(3, 4)",103.3.4,0.047074,https://macrohet.s3.eu-west-2.amazonaws.com/dM...,https://macrohet.s3.eu-west-2.amazonaws.com/dM...
2,0,355.510651,642.519592,1791.773679,3238.298745,0.0,0.0,0.0,20187.0,364.986389,...,RD1,CTRL,EC0,1,106,"(3, 4)",106.3.4,0.018784,https://macrohet.s3.eu-west-2.amazonaws.com/dM...,https://macrohet.s3.eu-west-2.amazonaws.com/dM...
3,0,1015.867676,793.773682,5119.973086,4000.619355,1.0,1.0,0.0,45891.0,384.419861,...,RD1,CTRL,EC0,1,72,"(3, 4)",72.3.4,0.012774,https://macrohet.s3.eu-west-2.amazonaws.com/dM...,https://macrohet.s3.eu-west-2.amazonaws.com/dM...
4,0,507.142792,923.500427,2555.999670,4654.442153,0.0,0.0,1.0,30221.0,374.314514,...,RD1,CTRL,EC0,1,50,"(3, 4)",50.3.4,0.107640,https://macrohet.s3.eu-west-2.amazonaws.com/dM...,https://macrohet.s3.eu-west-2.amazonaws.com/dM...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,0,586.682922,408.506195,2956.881929,2058.871223,1.0,1.0,1.0,32763.0,437.374390,...,WT,RIF,EC99,1,705,"(5, 7)",705.5.7,0.070339,https://macrohet.s3.eu-west-2.amazonaws.com/dM...,https://macrohet.s3.eu-west-2.amazonaws.com/dM...
119,0,1101.552734,586.640625,5551.825781,2956.668750,0.0,0.0,0.0,65290.0,366.450928,...,WT,RIF,EC99,1,619,"(5, 7)",619.5.7,0.028102,https://macrohet.s3.eu-west-2.amazonaws.com/dM...,https://macrohet.s3.eu-west-2.amazonaws.com/dM...
120,0,557.484619,649.688538,2809.722480,3274.430229,0.0,0.0,1.0,15791.0,382.747833,...,WT,RIF,EC99,1,91,"(5, 7)",91.5.7,0.074654,https://macrohet.s3.eu-west-2.amazonaws.com/dM...,https://macrohet.s3.eu-west-2.amazonaws.com/dM...
121,1,782.005249,989.538452,3941.306455,4987.273799,1.0,1.0,1.0,36830.0,363.714508,...,WT,RIF,EC99,1,749,"(5, 7)",749.5.7,0.075385,https://macrohet.s3.eu-west-2.amazonaws.com/dM...,https://macrohet.s3.eu-west-2.amazonaws.com/dM...


# Adding images


In [35]:
for index, row_ in tqdm(outliers.iterrows(), total = len(outliers)):
    unique_ID = row_['Unique ID']
    cell_ID, row, column = unique_ID.split('.') 
    t0 = row_['Time (hours)']
    time = [t0, -1]
    for t in time:
        output_fn = f'/mnt/DATA/macrohet/results/glimpses/interactive_plots/dMtb/glimpse_{cell_ID}.{row}.{column}_t{t}.png'
        if os.path.exists(output_fn):
            continue
        glimpse = tn_glimpse_maker(unique_ID, df, time=t)
        imsave(output_fn, glimpse)

  0%|          | 0/315 [00:00<?, ?it/s]

In [39]:
glimpse_root_dir = '/mnt/DATA/macrohet/results/glimpses/interactive_plots/dMtb/'

## Add new images

In [85]:
glimpse_fns_to_create = [fn for fn in fns if not os.path.exists(os.path.join(glimpse_root_dir, fn))]
for fn in tqdm(glimpse_fns_to_create, total = len(glimpse_fns_to_create)):
    output_fn = os.path.join(glimpse_root_dir, fn)
    if not os.path.exists(output_fn):
        unique_ID = fn.split('_')[1]
        time = int(fn.split('_')[-1].split('.png')[0].replace('t', ''))
        glimpse = visualise.tn_glimpse_maker(unique_ID, df, time, metadata)
        imsave(output_fn, glimpse)
        if time == -1:
            s3_file = outliers['glimpse_url_tn'][outliers['Unique ID']== unique_ID].iloc[0]
        else:
            s3_file = outliers['glimpse_url_t0'][outliers['Unique ID']== unique_ID].iloc[0]
        aws_tools.upload(output_fn, 'macrohet', s3_file)

  0%|          | 0/86 [00:00<?, ?it/s]

[INFO][2023/08/25 04:29:41 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5...
[INFO][2023/08/25 04:29:51 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/08/25 04:29:51 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5


https://macrohet.s3.eu-west-2.amazonaws.com/dMtb/dMtb_glimpses/glimpse_599.3.4_t0.png: Upload Successful


[INFO][2023/08/25 04:30:07 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5...
[INFO][2023/08/25 04:30:16 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/08/25 04:30:16 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5


https://macrohet.s3.eu-west-2.amazonaws.com/dMtb/dMtb_glimpses/glimpse_599.3.4_t-1.png: Upload Successful


...

https://macrohet.s3.eu-west-2.amazonaws.com/dMtb/dMtb_glimpses/glimpse_619.5.7_t-1.png: Upload Successful


[INFO][2023/08/25 05:10:35 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 7).h5...
[INFO][2023/08/25 05:10:44 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/08/25 05:10:44 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 7).h5


https://macrohet.s3.eu-west-2.amazonaws.com/dMtb/dMtb_glimpses/glimpse_875.6.7_t2.png: Upload Successful


[INFO][2023/08/25 05:11:00 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 7).h5...
[INFO][2023/08/25 05:11:10 PM] Loading segmentation (75, 6048, 6048)
[INFO][2023/08/25 05:11:10 PM] Closing HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(6, 7).h5


https://macrohet.s3.eu-west-2.amazonaws.com/dMtb/dMtb_glimpses/glimpse_875.6.7_t-1.png: Upload Successful


In [86]:
from macrohet import notify
notify.send_sms('images uploaded')